In [ ]:
import string
import os
import pandas as pd
import datetime
import numpy as np
import statsmodels.api as sm
import math
from scipy.linalg import svd
#import Functions as Fun
import scipy.linalg as la
import scipy.sparse.linalg as sla

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
X_all = pd.read_csv('../input/project2/X_cleaned.csv')
Y_all = pd.read_csv('../input/project2/Y_cleaned.csv')
Date_index = pd.read_csv('../input/project2/Date_list.csv')
Date_index['DATE']=pd.to_datetime(Date_index['DATE'])

In [ ]:
start_date = pd.to_datetime('1957-03-01')
end_date =  start_date + pd.offsets.DateOffset(years=18)
ind_training = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)
ytrain = Y_all[ind_training]
xtrain = X_all[ind_training]

In [ ]:
start_date =  end_date
end_date = start_date + pd.offsets.DateOffset(years=12)
ind_vali = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)
ytest = Y_all[ind_vali]
xtest= X_all[ind_vali]

In [ ]:
start_date =  end_date
end_date = start_date + pd.offsets.DateOffset(years=30)
ind_test = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)
#print(type(ind_test))
yoos = Y_all[ind_test]
xoos= X_all[ind_test]

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
#######Monthly Demean %%%
ytrain_demean= ytrain - np.mean(ytrain);
ytest_demean=ytest-np.mean(ytest);
mtrain=np.mean(ytrain);
mtest=np.mean(ytest);

sd=np.zeros(len(xtrain.columns)) #dim of sd? sd for each characteristics
for i in range(len(xtrain.columns)):
    s=np.std(xtrain.iloc[:,[i]])
    colnames = xtrain.columns
    if s.values > 0:
        colname = colnames[i]
        xtrain.loc[:,colname]=xtrain[colname]/s.values
        xtest.loc[:,colname]=xtest.loc[:,colname]/s.values
        xoos.loc[:,colname]=xoos.loc[:,colname]/s.values
        sd[i]=s.values
    #### Start to Train %%%
    #model fitting

lamv=np.arange(-2,4,0.1)
alpha=1
r=np.zeros((3,len(lamv)))
y_ret=yoos['RET'].reset_index(drop=True)
y_ret1=ytrain['RET'].reset_index(drop=True)
y_ret2=ytest['RET'].reset_index(drop=True)


In [ ]:
for j in range(len(lamv)):
    l2=10**lamv[j]
    ridge = Ridge(alpha=l2)
    yhatres= ridge.fit(xtrain,ytrain_demean)
    
    yhatbig1= yhatres.predict(xtest)+ mtrain.values
    yhatbig1=pd.Series(yhatbig1.flatten())
    r[0,j]=1-sum((yhatbig1-y_ret2)**2)/sum((y_ret2-mtrain.values)**2) 
    
    yhatbig1=yhatres.predict(xtrain) + mtrain.values
    yhatbig1=pd.Series(yhatbig1.flatten())
    r[1,j]=1-(yhatbig1-y_ret1).pow(2).sum()/sum(pow(y_ret1-mtrain.values,2))
    
    yhatbig1=yhatres.predict(xoos) + mtrain.values
    yhatbig1=pd.Series(yhatbig1.flatten())
    r[2,j]=1-sum((yhatbig1-y_ret)**2)/sum((y_ret-mtrain.values)**2)


r2_oos=r[2,np.argmax(r[0,:])]
r2_is=r[1,np.argmax(r[0,:])]
l2=10**lamv[np.argmax(r[0,:])]
print('r2_oos:'+np.str0(r2_oos))

In [ ]:
for i in range(len(X_all.columns)):
        xtrain_1=xtrain.drop(columns=[X_all.columns[i]])
        xtest_1=xtest.drop(columns=[X_all.columns[i]])
        xoos_1= xoos.drop(columns=[X_all.columns[i]])
        
        lamv=np.arange(-2,4,0.1)
        alpha=1
        r=np.zeros((3,len(lamv)))
        y_ret=yoos['RET'].reset_index(drop=True)
        y_ret1=ytrain['RET'].reset_index(drop=True)
        y_ret2=ytest['RET'].reset_index(drop=True)


        for j in range(len(lamv)):
            l2=10**lamv[j]
            ridge = Ridge(alpha=l2)
            yhatres= ridge.fit(xtrain_1,ytrain_demean)
            
            yhatbig1= yhatres.predict(xtest_1)+ mtrain.values
            yhatbig1=pd.Series(yhatbig1.flatten())
            r[0,j]=1-sum((yhatbig1-y_ret2)**2)/sum((y_ret2-mtrain.values)**2)
            
            yhatbig1=yhatres.predict(xtrain_1) + mtrain.values
            yhatbig1=pd.Series(yhatbig1.flatten())
            r[1,j]=1-(yhatbig1-y_ret1).pow(2).sum()/sum(pow(y_ret1-mtrain.values,2))
            
            yhatbig1=yhatres.predict(xoos_1) + mtrain.values
            yhatbig1=pd.Series(yhatbig1.flatten())
            r[2,j]=1-sum((yhatbig1-y_ret)**2)/sum((y_ret-mtrain.values)**2)
            

        r2_oos_1=r[2,np.argmax(r[0,:])]
        l2=10**lamv[np.argmax(r[0,:])]
        
        d=r2_oos-r2_oos_1
        print(X_all.columns[i]+':'+np.str0(d))


<!--
beta:-4.196077164298913
betasq:-20.017853973033198
chmom:-13.869830809933774
dolvol:674.645307093182
idiovol:55.8835101347795
indmom:181.8716420403871
mom1m:-2391.870468121262
mom6m:83.69082459850324
mom12m:49.11506304379873
mom36m:47.74877162298071
pricedelay:18.88163791894476
turn:-3.8489216333036893
absacc:89.85183998235152
acc:60.30084910678852
age:232.9500460492054
agr:21.123519215077977
cashdebt:159.8668742142254
cashpr:238.1565357313957
cfp:35.31192071102123
cfp_ia:-30.413314909746987
chatoia:209.44781613680243
chcsho:78.04607184873021
chempia:60.99763918566896
chinv:-107.41713972100843
chpmia:57.749147206042835
convind:-45.792169259388174
currat:115.72544283904426
depr:-32.082533087886986
divi:-18.789584441306943
divo:125.57451701069658
dy:593.3677316693938
egr:69.90587310535193
ep:139.77067957504187
gma:8.14839935668715
grcapx:626.8969955528955
grltnoa:-38.768435354024405
herf:15.19522776901431
hire:-166.89846328249405
invest:-76.01871651178226
lev:502.41574192197004
lgr:-52.39464602465159
mve_ia:-89.80064833176584
operprof:-6.021213490785158
orgcap:-210.0081194275699
pchcapx_ia:28.34184716216987
pchcurrat:-91.4344400577611
pchdepr:-30.652838289097417
pchgm_pchsale:118.69969519779261
pchquick:-32.67463321157993
pchsale_pchinvt:-44.545291228292626
pchsale_pchrect:-76.22169891990052
pchsale_pchxsga:60.499765452652355
pchsaleinv:-32.94777329760109
pctacc:172.93595188668405
ps:-25.300703159977274
quick:40.51706448781624
rd:18.88068081792153
rd_mve:115.2990804429428
rd_sale:44.51519569703669
realestate:0.0
roic:-4.5379351619121735
salecash:-4.583299838297535
saleinv:-27.48258093723416
salerec:-4.229880833765492
secured:0.0
securedind:0.0
sgr:107.94584636765649
sin:0.0
sp:128.685464692011
tang:526.925481190483
tb:-10.843630101277085
aeavol:148.0985291078614
cash:-1151.8465734600613
chtx:1627.8419003401505
cinvest:3748.597686090754
ear:-84.84174972368783
nincr:-62.91191619435267
roaq:382.3567176011711
roavol:-827.4262426019777
roeq:-554.7557873754195
rsup:-281.3735012145771
stdacc:-14590.373214082963
stdcf:-24451.606329504764
ms:-292.69271072329866
baspread:855.1387758443307
ill:-301.19509004761494
maxret:67.75045178907749
retvol:712.826657675585
std_dolvol:123.57874405625626
std_turn:-361.88497287268547
zerotrade:143.96634013445873
bm:433.5498843666719
bm_ia:108.7504320854423
sic2_10:0.0
sic2_13:135.88794020145724
sic2_15:353.0432767190141
sic2_17:0.0
sic2_20:17.636425607161073
sic2_22:17.98834225419705
sic2_23:371.3908806231775
sic2_24:141.35341669246554
sic2_25:0.0
sic2_26:69.9692062444883
sic2_27:23.385667960887076
sic2_28:243.77939655012597
sic2_29:-191.74906103934336
sic2_30:-87.24047058444557
sic2_32:94.22552696445928
sic2_33:-7.977295418168069
sic2_34:31.019423529134656
sic2_35:80.08055244494608
sic2_36:52.378799401267315
sic2_37:43.10794804905163
sic2_38:113.66365818832128
sic2_39:48.81558140151901
sic2_40:-6.654723651125096
sic2_41:0.0
sic2_42:0.0
sic2_44:107.7650885529365
sic2_45:-12.41655333154631
sic2_47:0.0
sic2_48:0.0
sic2_49:4.888183194110752
sic2_50:22.10089544759103
sic2_51:0.0
sic2_52:0.0
sic2_53:0.0
sic2_54:7.615614876536711
sic2_56:0.0
sic2_57:0.0
sic2_58:-22.001768900510797
sic2_59:79.2642332446485
sic2_60:54.6169499571115
sic2_61:56.66219064636243
sic2_62:4.215477459751128
sic2_63:-19.38716955689597
sic2_64:0.0
sic2_65:240.17341550799028
sic2_67:-98.80909630588576
sic2_70:0.0
sic2_72:0.0
sic2_73:-2.065557213005377
sic2_75:0.870048494747607
sic2_78:52.45424535443453
sic2_79:0.0
sic2_80:0.0
sic2_87:-62.89841065360088
sic2_99:0.0